In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from config import *

synthetic_dir = "../data/carla"
sim2real_dir = "../data/carla-cityscapes"

In [11]:
from dataset import *

TOWN_LIST = ["Town01"]

synthetic_dataset = CarlaDataset(
  base_dir=synthetic_dir,
  townslist=TOWN_LIST,
  image_size=IMAGE_SIZE,
  use_imagenet_norm=USE_IMAGENET_NORM,
  sequence_size=SEQUENCE_SIZE
)

sim2real_dataset = CarlaDataset(
  base_dir=sim2real_dir,
  townslist=TOWN_LIST,
  image_size=IMAGE_SIZE,
  use_imagenet_norm=USE_IMAGENET_NORM,
  sequence_size=SEQUENCE_SIZE
)

[CarlaDataset] Loading Town01: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


In [12]:
import torch.nn as nn
from torchvision import models

model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

/home/pavlos/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pavlos/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [13]:
def stack_images(x):
  x = x.permute(0, 2, 1, 3, 4)      # -> (B, T, C, H, W)
  x = x.reshape(-1, 3, 224, 224)    # -> (B*T, C, H, W)
  return x

In [14]:
import numpy as np
from tqdm.notebook import tqdm


def extract_features(dataloader):
  features = []
  for data in (t := tqdm(dataloader)):
    inputs, _ = data
    rgb_left = inputs['rgb_left']
    rgb_front = inputs['rgb_front']
    rgb_right = inputs['rgb_right']
    
    for rgb in [rgb_left, rgb_front, rgb_right]:
      x = stack_images(rgb)
      with torch.no_grad():
        feat = model(x).squeeze().numpy()  # (BS, 2048)
      features.append(feat)

  features = np.concatenate(features, axis=0)
  return features

In [15]:
from torch.utils.data import DataLoader

synthetic_loader = DataLoader(synthetic_dataset, batch_size=64, num_workers=8)
sim2real_loader  = DataLoader(sim2real_dataset, batch_size=64, num_workers=8)

synthetic_feats = extract_features(synthetic_loader)
print(f"Extracted synthetic features, shape: {synthetic_feats.shape}")
sim2real_feats = extract_features(sim2real_loader)
print(f"Extracted sim2real features, shape: {sim2real_feats.shape}")

  0%|          | 0/156 [00:00<?, ?it/s]

Extracted synthetic features, shape: (239472, 2048)


  0%|          | 0/156 [00:00<?, ?it/s]

Extracted sim2real features, shape: (239472, 2048)


In [19]:
def kl_divergence_gaussian(mu1, cov1, mu2, cov2):
  """
  KL(N1||N2) for multivariate Gaussian
  """
  dim = mu1.shape[0]
  cov2_inv = np.linalg.inv(cov2)
  term1 = np.trace(cov2_inv @ cov1)
  term2 = (mu2 - mu1).T @ cov2_inv @ (mu2 - mu1)
  term3 = np.log(np.linalg.det(cov2) / np.linalg.det(cov1) + 1e-10)
  return 0.5 * (term1 + term2 - dim + term3)

muA, covA = np.mean(synthetic_feats, axis=0), np.cov(synthetic_feats, rowvar=False)
muB, covB = np.mean(sim2real_feats, axis=0), np.cov(sim2real_feats, rowvar=False)

print(f"Synthetic Mean: {muA.shape}, Cov: {covA.shape}")
print(f"Sim2Real Mean: {muB.shape}, Cov: {covB.shape}")

print(muA)
print(covA)
print(muB)
print(covB)

kl_AB = kl_divergence_gaussian(muA, covA, muB, covB)
kl_BA = kl_divergence_gaussian(muB, covB, muA, covA)

print(f"KL(Synthetic || Sim2Real): {kl_AB}")
print(f"KL(Sim2Real || Synthetic): {kl_BA}")

Synthetic Mean: (2048,), Cov: (2048, 2048)
Sim2Real Mean: (2048,), Cov: (2048, 2048)
[0.5348786  2.0857584  0.21238399 ... 0.4052946  0.18515387 0.608695  ]
[[ 1.52470133e-01 -1.18290947e-01  3.87678477e-02 ... -1.24830841e-02
   2.73709633e-02  4.25964260e-02]
 [-1.18290947e-01  6.55299276e-01 -1.12746332e-02 ... -6.83057708e-05
  -7.49974575e-02  5.69644272e-02]
 [ 3.87678477e-02 -1.12746332e-02  3.45971205e-02 ... -5.43802640e-03
   2.92125080e-03  2.16460424e-02]
 ...
 [-1.24830841e-02 -6.83057708e-05 -5.43802640e-03 ...  4.66779225e-02
   2.35573303e-03 -1.88924599e-03]
 [ 2.73709633e-02 -7.49974575e-02  2.92125080e-03 ...  2.35573303e-03
   2.90976790e-02  2.20822646e-03]
 [ 4.25964260e-02  5.69644272e-02  2.16460424e-02 ... -1.88924599e-03
   2.20822646e-03  9.81217468e-02]]
[0.38277948 2.7284997  0.21921875 ... 0.28532958 0.01794278 0.77807134]
[[ 0.09042121 -0.05784214  0.02579608 ... -0.00455963  0.00238433
   0.06478593]
 [-0.05784214  0.24741308 -0.01372178 ... -0.00504667 

/tmp/ipykernel_51624/3468491119.py:9: RuntimeWarning: invalid value encountered in scalar divide
  term3 = np.log(np.linalg.det(cov2) / np.linalg.det(cov1) + 1e-10)


KL(Synthetic || Sim2Real): nan
KL(Sim2Real || Synthetic): nan


In [21]:
import torch
from torch.distributions import Categorical, kl_divergence

def features_to_distribution(features, bins=100):
  """
  Convert high-dimensional features into a 1D histogram-based distribution.
  We flatten all features across samples and bin them.
  """
  flat = features.flatten()
  hist, edges = np.histogram(flat, bins=bins, density=True)
  dist = hist / np.sum(hist)  # normalize to probability distribution
  return dist

def safe_dist(dist, eps=1e-8):
  dist = np.asarray(dist, dtype=np.float64)
  dist = dist + eps        # avoid zeros
  dist = dist / dist.sum() # renormalize
  return dist

def kl_torch(distP, distQ):
  """
  KL(P||Q) using torch distributions
  """
  P = Categorical(probs=torch.tensor(distP, dtype=torch.float32))
  Q = Categorical(probs=torch.tensor(distQ, dtype=torch.float32))
  return kl_divergence(P, Q).item()

dist_synthetic = features_to_distribution(synthetic_feats, bins=200)
dist_sim2real  = features_to_distribution(sim2real_feats, bins=200)

dist_synthetic = safe_dist(dist_synthetic)
dist_sim2real  = safe_dist(dist_sim2real)

kl_AB = kl_torch(dist_synthetic, dist_sim2real)
kl_BA = kl_torch(dist_sim2real, dist_synthetic)

print(f"KL(Synthetic || Sim2Real) [Histogram]: {kl_AB}")
print(f"KL(Sim2Real || Synthetic) [Histogram]: {kl_BA}")
print("Symmetric KL:", 0.5 * (kl_AB + kl_BA))

KL(Synthetic || Sim2Real) [Histogram]: 0.022729923948645592
KL(Sim2Real || Synthetic) [Histogram]: 0.025857236236333847
Symmetric KL: 0.02429358009248972


In [22]:
np.save("dist_synthetic.npy", dist_synthetic)
np.save("dist_sim2real.npy", dist_sim2real)